In [1]:
import logging
from typing import Dict
from typing import List, Optional
from typing import Tuple
import requests
from _datetime import datetime
from bs4 import BeautifulSoup
import threading
import re
import calendar
import json

In [2]:
IMDB_SEARCH_TITLE_URL = "https://www.imdb.com/search/title/"
IMDB_URL = "https://www.imdb.com"
HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9,ru;q=0.8",
    "cache-control": "max-age=0",
    "cookie": "uu=BCYhkhONRrPmyyxa5h6CCV5Sc8PkddDeFiSeg_uc9WBlOmUn48sTxd9jKWWgArGnZ6BQ5cTPSIn3%0D%0AyhTESbgq9a0-gNwkywfmoHL3pUH9Em1V6a7iEiGHeVs-jacIjARxhAZlxuWVX4DW1FvNm8L6yae_%0D%0AFQ%0D%0A; adblk=adblk_no; session-token=iYFlbLBfkhCCbXZy2fGrhCLI7rfgjbh5HLddZveNthYusXnc3bKkIv0DfByssxWmCMx+dSTTEZM/rJoxkNLUe9fmxOckf34SK03PLmV34RwA6XzR9cqCw82WyARUNuFIVyVf5+LvnY8aLKDalXmCx1lpyZHXoezmNWndE1ECUQa6QDxNXnqOEDRd3Ef7cwVF; csm-hit=tb:ZA4E08Q73G2PW9VDN6EE+s-2C6E1BKQ929VAJH2NEYA|1610473595525&t:1610473595525&adb:adblk_yes"}

MAX_FETCH_RESULTS = 1000
PAGE_SIZE = 50
MAX_FETCH_PAGES = int(MAX_FETCH_RESULTS / PAGE_SIZE)

In [3]:
def optional_float_to_str(f: float) -> str:
    return '' if f is None else str(f)

def optional_datetime_to_str(datetime_parameter: datetime) -> str:
    return '' if datetime_parameter is None else datetime_parameter.strftime("%Y-%m-%d")

In [4]:
class SearchParameters:

    def __str__(self) -> str:
        return """[
        title_types={},
        release_date_from={},
        release_date_to={},
        genres={},
        user_rating_from={},
        user_rating_to={},
        countries={}
        ]""".format(
            self.title_types,
            self.release_date_from,
            self.release_date_to,
            self.genres,
            self.user_rating_from,
            self.user_rating_to,
            self.countries)

    def __init__(self, title_types: List[str] = None, release_date_from: datetime = None,
                 release_date_to: datetime = None, genres: List[str] = None, user_rating_from: float = None,
                 user_rating_to: float = None, countries: List[str] = None) -> None:
        self.title_types = title_types
        self.release_date_from = release_date_from
        self.release_date_to = release_date_to
        self.genres = genres
        self.user_rating_from = user_rating_from
        self.user_rating_to = user_rating_to
        self.countries = countries

    def is_empty(self) -> bool:
        return self.title_types is None and \
               self.release_date_from is None and \
               self.release_date_to is None and \
               self.genres is None and \
               self.user_rating_from is None and \
               self.user_rating_to is None and \
               self.countries is None

In [5]:
class Actor:

    def __init__(self, real_name: str, character_name: str) -> None:
        self.real_name = real_name
        self.character_name = character_name


class MovieInfo:

    def __init__(self, title: str, genres: List[str], rating: Optional[float], actors: List[Actor],
                 details: Dict[str, str]) -> None:
        self.title = title
        self.genres = genres
        self.rating = rating
        self.actors = actors
        self.details = details


class SearchResult:

    def __init__(self, search_title: str, search_parameters: SearchParameters, movie_infos: List[MovieInfo]) -> None:
        self.search_title = search_title
        self.search_parameters = search_parameters
        self.movie_infos = movie_infos


In [6]:
class IMDBParser:

    def __init__(self, search_parameters: SearchParameters) -> None:
        self.search_parameters = search_parameters
        logging.info(
            "Initializing IMDBParser with search_parameters={}".format(search_parameters))

    def get_search_result(self) -> SearchResult:
        logging.info("Getting search result from IMDB site with parameters = {}".format(self.search_parameters))
        query_parameters = {}
        if self.search_parameters.countries is not None:
            query_parameters["countries"] = ",".join(self.search_parameters.countries)
        if self.search_parameters.title_types is not None:
            query_parameters["title_type"] = ",".join(self.search_parameters.title_types)
        if self.search_parameters.genres is not None:
            query_parameters["genres"] = ",".join(self.search_parameters.genres)
        if self.search_parameters.user_rating_from is not None or self.search_parameters.user_rating_to is not None:
            query_parameters["user_rating"] = "{},{}".format(
                optional_float_to_str(self.search_parameters.user_rating_from),
                optional_float_to_str(self.search_parameters.user_rating_to))
        if self.search_parameters.release_date_from is not None or self.search_parameters.release_date_to is not None:
            query_parameters["release_date"] = "{},{}".format(
                optional_datetime_to_str(self.search_parameters.release_date_from),
                optional_datetime_to_str(self.search_parameters.release_date_to))
        movie_infos = []
        title = None
        for page_number in range(MAX_FETCH_PAGES):
            query_parameters["start"] = 1 + page_number * PAGE_SIZE
            r = requests.get(IMDB_SEARCH_TITLE_URL, params=query_parameters, headers=HEADERS)
            soup = BeautifulSoup(r.text, "html.parser")
            h3_tags = soup.find_all("h3", attrs={"class": "lister-item-header"})
            title = soup.find("h1").text.strip()
            length = len(h3_tags)
            logging.info("Received {} movies".format(length))
            if len(h3_tags) == 0:
                break
            page_threads = []
            for idx, h3_tag in enumerate(h3_tags):
                page_thread = threading.Thread(
                    target=lambda: self.append_movie_infos(h3_tag, idx, length, movie_infos, page_number))
                page_threads.append(page_thread)
                page_thread.start()
            for page_thread in page_threads:
                page_thread.join()
        return SearchResult(title, self.search_parameters, movie_infos)

    def append_movie_infos(self, h3_tag, idx, length, movie_infos, page_number) -> None:
        print("Getting info for movie {} out of {} on page {}".format(idx + 1, length, page_number))
        logging.info("Getting info for movie {} out of {} on page {}".format(idx + 1, length, page_number))
        a_tag = h3_tag.find("a")
        href = a_tag.get("href")
        try:
            movie_infos.append(self.get_movie_info(IMDB_URL + href))
        except Exception as e:
            print("Error on getting movie info " + str(e))
            logging.error(e)
            return
        print("Finished getting info for movie {} out of {} on page {}".format(idx + 1, length, page_number))

    @staticmethod
    def get_movie_info(link: str) -> MovieInfo:
        logging.info("Sending request {}".format(link))
        r = requests.get(link, headers=HEADERS)
        soup = BeautifulSoup(r.text, "html.parser")
        title = soup.find("h1").text.strip()
        logging.info("Received info for movie {}".format(title))
        genres = list(set(
            map(lambda tag: tag.text.strip(),
                filter(lambda tag: tag.has_attr("href") and "genres" in tag.get("href"),
                       soup.find_all("a")))))
        rating_tag = soup.find("span", attrs={"itemprop": "ratingValue"})
        rating = float(rating_tag.text) if rating_tag is not None else None
        actors = []
        cast_infos = map(lambda tag: tag.find_parent(), soup.find_all("td", attrs={"class": "primary_photo"}))
        for cast_info in cast_infos:
            a_tags = cast_info.find_all("a")
            real_name = \
                list(map(lambda tag: tag.text.strip(), filter(lambda tag: "name" in tag.get("href"), a_tags)))[1]
            character_name_array = list(
                map(lambda tag: tag.text.strip(), filter(lambda tag: "characters" in tag.get("href"), a_tags)))
            character_name = "" if len(character_name_array) == 0 else character_name_array[0]
            actors.append(Actor(real_name, character_name))
        details = list(map(
            lambda tag: tag.text.strip().replace("\n", "").replace("See more »", "").replace(
                "Show more on  IMDbPro »",
                ""),
            soup.find(text="Details").find_parent().find_parent().find_all("div",
                                                                           attrs={
                                                                               "class": "txt-block"})))
        details_dict = {}
        for detail in details:
            detail_split = str(detail).split(":")
            if detail_split[0] == "":
                continue
            details_dict[detail_split[0]] = ":".join(detail_split[1:]).strip()
        return MovieInfo(title, genres, rating, actors, details_dict)

    @staticmethod
    def get_available_title_types() -> List[Tuple[str, str]]:
        logging.info("Getting available title types from IMDB site")
        r = requests.get(IMDB_SEARCH_TITLE_URL, headers=HEADERS)
        soup = BeautifulSoup(r.text, "html.parser")
        result = list()
        for input_tag in soup.find_all("input", attrs={"name": "title_type"}):
            input_tag_id = input_tag.get("id")
            label_tag = soup.find("label", attrs={"for": input_tag_id})
            title_value = input_tag.get("value")
            title_label = label_tag.text
            result.append((title_label, title_value))
        logging.info("Received following title types from IMDB {}".format(result))
        return result

    @staticmethod
    def get_available_genres() -> List[Tuple[str, str]]:
        logging.info("Getting available genres from IMDB site")
        r = requests.get(IMDB_SEARCH_TITLE_URL, headers=HEADERS)
        soup = BeautifulSoup(r.text, "html.parser")
        result = list()
        for input_tag in soup.find_all("input", attrs={"name": "genres"}):
            input_tag_id = input_tag.get("id")
            label_tag = soup.find("label", attrs={"for": input_tag_id})
            genre_value = input_tag.get("value")
            genre_label = label_tag.text
            result.append((genre_label, genre_value))
        logging.info("Received following genres from IMDB {}".format(result))
        return result

    @staticmethod
    def get_available_countries() -> List[Tuple[str, str]]:
        logging.info("Getting available countries from IMDB site")
        r = requests.get(IMDB_SEARCH_TITLE_URL, headers=HEADERS)
        soup = BeautifulSoup(r.text, "html.parser")
        result = list()
        select_tag = soup.find("select", attrs={"name": "countries"})
        for option_tag in select_tag.find_all():
            genre_value = option_tag.get("value")
            genre_label = option_tag.text
            result.append((genre_label, genre_value))
        logging.info("Received following countries from IMDB {}".format(result))
        return result

In [7]:
root_logger = logging.getLogger()
root_logger.setLevel(logging.DEBUG)
handler = logging.FileHandler('parser_log.log', 'a', 'utf8')
handler.setFormatter(
    logging.Formatter(fmt='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s', datefmt='%H:%M:%S'))
root_logger.addHandler(handler)

In [8]:
def get_user_title_types() -> Optional[List[str]]:
    print("Select title types")
    title_types = IMDBParser.get_available_title_types()
    for idx, title_type in enumerate(title_types):
        print("{}) {}".format(idx, title_type[0]))
    while True:
        logging.info("Asking user to select title types")
        inp = input("Enter space-separated values or leave empty for empty list: ").strip()
        if not inp:
            logging.info("User didn't select any title types")
            return None
        split_input = set(inp.split())
        result = []
        try:
            for str_value in split_input:
                int_value = int(str_value)
                result.append(title_types[int_value][1])
        except Exception as e:
            logging.error(e)
            print("Wrong input. Try again.")
            continue
        logging.info("User selected following title types {}".format(result))
        return result


def get_user_genres() -> Optional[List[str]]:
    print("Select genres")
    genres = IMDBParser.get_available_genres()
    for idx, genre in enumerate(genres):
        print("{}) {}".format(idx, genre[0]))
    while True:
        logging.info("Asking user to select genres")
        inp = input("Enter space-separated values or leave empty for empty list: ").strip()
        if not inp:
            logging.info("User didn't select any genre")
            return None
        split_input = set(inp.split())
        result = []
        try:
            for str_value in split_input:
                int_value = int(str_value)
                result.append(genres[int_value][1])
        except Exception as e:
            logging.error(e)
            print("Wrong input. Try again.")
            continue
        logging.info("User selected following genres {}".format(result))
        return result


def get_user_countries() -> Optional[List[str]]:
    print("Select countries")
    countries = IMDBParser.get_available_countries()
    for idx, country in enumerate(countries):
        print("{}) {}".format(idx, country[0]))
    while True:
        logging.info("Asking user to select countries")
        inp = input("Enter space-separated values or leave empty for empty list: ").strip()
        if not inp:
            logging.info("User didn't select any country")
            return None
        split_input = set(inp.split())
        result = []
        try:
            for str_value in split_input:
                int_value = int(str_value)
                result.append(countries[int_value][1])
        except Exception as e:
            logging.error(e)
            print("Wrong input. Try again.")
            continue
        logging.info("User selected following countries {}".format(result))
        return result


year_month_day_pattern = re.compile("^(\\d{4})-(\\d{2})-(\\d{2})$")
year_month_pattern = re.compile("^(\\d{4})-(\\d{2})$")
year_pattern = re.compile("^(\\d{4})$")


def get_user_release_date() -> Tuple[Optional[datetime], Optional[datetime]]:
    release_date_from = get_user_date("release date from", False)
    release_date_to = get_user_date("release date to", True)
    return release_date_from, release_date_to


def get_user_date(description: str, max_day: bool) -> Optional[datetime]:
    date = None
    while True:
        logging.info("Asking user to enter " + description)
        try:
            date_str = input(
                "Enter {} in one of following formats: YYYY-MM-DD, YYYY-MM, YYYY or leave empty: ".format(
                    description)).strip()
            if not date_str:
                logging.info("User entered empty " + description)
                break
            match = year_month_day_pattern.match(date_str)
            if match:
                year = match.group(1)
                month = match.group(2)
                day = match.group(3)
                date = datetime(int(year), int(month), int(day))
                logging.info("User entered {} {}".format(description, date))
                break
            match = year_month_pattern.match(date_str)
            if match:
                year = match.group(1)
                month = match.group(2)
                if not max_day:
                    date = datetime(int(year), int(month), 1)
                else:
                    max_day = calendar.monthrange(int(year), int(month))[1]
                    date = datetime(int(year), int(month), max_day)
                logging.info("User entered {} {}".format(description, date))
                break
            match = year_pattern.match(date_str)
            if match:
                year = match.group(1)
                if not max_day:
                    date = datetime(int(year), 1, 1)
                else:
                    date = datetime(int(year), 12, 31)
                logging.info("User entered {} {}".format(description, date))
                break
            raise Exception('Wrong input {}'.format(date_str))
        except Exception as e:
            logging.error(e)
            print("Wrong input. Try again.")
            continue
    return date


def get_user_rating(description: str) -> Optional[float]:
    while True:
        logging.info("Asking user to enter " + description)
        rating_str = input("Please enter {} in range from 1.0 to 10.0: ".format(description)).strip()
        if not rating_str:
            logging.info("User entered empty " + description)
            return None
        try:
            rating_float = float(rating_str)
            if rating_float < 1.0 or rating_float > 10.0:
                raise Exception("Rating should be in range from 1.0 to 10.0")
            logging.info("User entered {} {}".format(description, rating_float))
            return rating_float
        except Exception as e:
            logging.error(e)
            print("Wrong input. Try again")
            continue


def get_user_ratings() -> Tuple[Optional[float], Optional[float]]:
    user_rating_from = get_user_rating("user rating from")
    user_rating_to = get_user_rating("user rating to")
    return user_rating_from, user_rating_to

In [9]:
def main() -> None:
    while True:
        logging.info("Asking user to fill parameters")
        title_types = get_user_title_types()
        genres = get_user_genres()
        countries = get_user_countries()
        release_date_from, release_date_to = get_user_release_date()
        user_rating_from, user_rating_to = get_user_ratings()
        search_parameters = SearchParameters(title_types, release_date_from, release_date_to, genres, user_rating_from,
                                             user_rating_to, countries)
        if search_parameters.is_empty():
            logging.info("User didn't enter any parameters")
            print("Search parameters can't be empty. Fill at least one parameter.")
            continue
        try:
            imdb_parser = IMDBParser(search_parameters)
            search_result = imdb_parser.get_search_result()
            json_str = json.dumps(search_result.__dict__, default=lambda x: x.__dict__, ensure_ascii=False,
                                  indent=4).encode("utf8")
            with open("output.json", "wb") as f:
                f.write(json_str)

        except Exception as e:
            logging.error(e)
            print("Error during parsing " + str(e))
        break


if __name__ == "__main__":
    main()

Select title types
0) Feature Film
1) TV Movie
2) TV Series
3) TV Episode
4) TV Special
5) Mini-Series
6) Documentary
7) Video Game
8) Short Film
9) Video
10) TV Short
Enter space-separated values or leave empty for empty list: 0
Select genres
0) Action
1) Adventure
2) Animation
3) Biography
4) Comedy
5) Crime
6) Documentary
7) Drama
8) Family
9) Fantasy
10) Film-Noir
11) Game-Show
12) History
13) Horror
14) Music
15) Musical
16) Mystery
17) News
18) Reality-TV
19) Romance
20) Sci-Fi
21) Sport
22) Talk-Show
23) Thriller
24) War
25) Western
Enter space-separated values or leave empty for empty list: 0
Select countries
0) Afghanistan
1) Åland Islands
2) Albania
3) Algeria
4) American Samoa
5) Andorra
6) Angola
7) Anguilla
8) Antarctica
9) Antigua and Barbuda
10) Argentina
11) Armenia
12) Aruba
13) Australia
14) Austria
15) Azerbaijan
16) Bahamas
17) Bahrain
18) Bangladesh
19) Barbados
20) Belarus
21) Belgium
22) Belize
23) Benin
24) Bermuda
25) Bhutan
26) Bolivia
27) Bonaire, Sint Eustat

Finished getting info for movie 6 out of 50 on page 0
Finished getting info for movie 36 out of 50 on page 0
Finished getting info for movie 12 out of 50 on page 0
Finished getting info for movie 17 out of 50 on page 0
Finished getting info for movie 45 out of 50 on page 0
Finished getting info for movie 42 out of 50 on page 0
Finished getting info for movie 20 out of 50 on page 0Finished getting info for movie 14 out of 50 on page 0

Finished getting info for movie 38 out of 50 on page 0Finished getting info for movie 48 out of 50 on page 0

Finished getting info for movie 26 out of 50 on page 0
Finished getting info for movie 46 out of 50 on page 0
Finished getting info for movie 21 out of 50 on page 0
Finished getting info for movie 24 out of 50 on page 0
Finished getting info for movie 37 out of 50 on page 0
Finished getting info for movie 32 out of 50 on page 0
Finished getting info for movie 35 out of 50 on page 0
Finished getting info for movie 25 out of 50 on page 0
Finished ge

Finished getting info for movie 46 out of 50 on page 2
Finished getting info for movie 28 out of 50 on page 2
Finished getting info for movie 42 out of 50 on page 2
Finished getting info for movie 33 out of 50 on page 2
Finished getting info for movie 15 out of 50 on page 2
Finished getting info for movie 21 out of 50 on page 2
Finished getting info for movie 36 out of 50 on page 2
Finished getting info for movie 47 out of 50 on page 2
Finished getting info for movie 29 out of 50 on page 2
Finished getting info for movie 37 out of 50 on page 2
Finished getting info for movie 50 out of 50 on page 2Finished getting info for movie 32 out of 50 on page 2

Finished getting info for movie 3 out of 50 on page 2
Finished getting info for movie 2 out of 50 on page 2
Finished getting info for movie 31 out of 50 on page 2
Finished getting info for movie 48 out of 50 on page 2
Finished getting info for movie 5 out of 50 on page 2
Finished getting info for movie 22 out of 50 on page 2
Finished gett

Getting info for movie 45 out of 50 on page 4
Getting info for movie 46 out of 50 on page 4
Getting info for movie 47 out of 50 on page 4
Getting info for movie 48 out of 50 on page 4
Getting info for movie 49 out of 50 on page 4
Getting info for movie 50 out of 50 on page 4
Finished getting info for movie 3 out of 50 on page 4
Finished getting info for movie 19 out of 50 on page 4
Finished getting info for movie 7 out of 50 on page 4Finished getting info for movie 16 out of 50 on page 4

Finished getting info for movie 20 out of 50 on page 4Finished getting info for movie 4 out of 50 on page 4

Finished getting info for movie 47 out of 50 on page 4
Finished getting info for movie 40 out of 50 on page 4
Finished getting info for movie 12 out of 50 on page 4Finished getting info for movie 17 out of 50 on page 4
Finished getting info for movie 1 out of 50 on page 4
Finished getting info for movie 15 out of 50 on page 4

Finished getting info for movie 48 out of 50 on page 4
Finished gett

Finished getting info for movie 8 out of 50 on page 6
Finished getting info for movie 13 out of 50 on page 6
Finished getting info for movie 43 out of 50 on page 6
Finished getting info for movie 3 out of 50 on page 6
Finished getting info for movie 49 out of 50 on page 6
Finished getting info for movie 12 out of 50 on page 6
Finished getting info for movie 35 out of 50 on page 6Finished getting info for movie 48 out of 50 on page 6Finished getting info for movie 23 out of 50 on page 6


Finished getting info for movie 27 out of 50 on page 6
Finished getting info for movie 47 out of 50 on page 6Finished getting info for movie 30 out of 50 on page 6

Finished getting info for movie 4 out of 50 on page 6Finished getting info for movie 28 out of 50 on page 6

Finished getting info for movie 16 out of 50 on page 6
Finished getting info for movie 20 out of 50 on page 6
Finished getting info for movie 18 out of 50 on page 6
Finished getting info for movie 6 out of 50 on page 6
Finished getti

...

Getting info for movie 42 out of 50 on page 12Getting info for movie 43 out of 50 on page 12

Getting info for movie 44 out of 50 on page 12
Getting info for movie 45 out of 50 on page 12
Getting info for movie 46 out of 50 on page 12
Getting info for movie 47 out of 50 on page 12
Getting info for movie 48 out of 50 on page 12Getting info for movie 49 out of 50 on page 12

Getting info for movie 50 out of 50 on page 12
Finished getting info for movie 5 out of 50 on page 12
Finished getting info for movie 10 out of 50 on page 12Finished getting info for movie 2 out of 50 on page 12
Finished getting info for movie 1 out of 50 on page 12

Finished getting info for movie 13 out of 50 on page 12
Finished getting info for movie 6 out of 50 on page 12
Finished getting info for movie 9 out of 50 on page 12
Finished getting info for movie 17 out of 50 on page 12
Finished getting info for movie 21 out of 50 on page 12
Finished getting info for movie 46 out of 50 on page 12
Finished getting info 

Finished getting info for movie 35 out of 50 on page 13
Getting info for movie 1 out of 50 on page 14
Getting info for movie 2 out of 50 on page 14
Getting info for movie 3 out of 50 on page 14Getting info for movie 4 out of 50 on page 14

Getting info for movie 5 out of 50 on page 14
Getting info for movie 6 out of 50 on page 14
Getting info for movie 7 out of 50 on page 14
Getting info for movie 8 out of 50 on page 14
Getting info for movie 9 out of 50 on page 14
Getting info for movie 10 out of 50 on page 14
Getting info for movie 11 out of 50 on page 14
Getting info for movie 12 out of 50 on page 14Getting info for movie 13 out of 50 on page 14

Getting info for movie 14 out of 50 on page 14
Getting info for movie 15 out of 50 on page 14
Getting info for movie 16 out of 50 on page 14
Getting info for movie 17 out of 50 on page 14
Getting info for movie 18 out of 50 on page 14
Getting info for movie 19 out of 50 on page 14
Getting info for movie 20 out of 50 on page 14Getting info f

Finished getting info for movie 13 out of 50 on page 15Finished getting info for movie 22 out of 50 on page 15Finished getting info for movie 36 out of 50 on page 15


Finished getting info for movie 21 out of 50 on page 15Finished getting info for movie 46 out of 50 on page 15

Finished getting info for movie 29 out of 50 on page 15
Finished getting info for movie 14 out of 50 on page 15
Finished getting info for movie 8 out of 50 on page 15Finished getting info for movie 38 out of 50 on page 15

Finished getting info for movie 45 out of 50 on page 15
Finished getting info for movie 18 out of 50 on page 15
Finished getting info for movie 48 out of 50 on page 15
Finished getting info for movie 43 out of 50 on page 15Finished getting info for movie 41 out of 50 on page 15
Finished getting info for movie 6 out of 50 on page 15

Finished getting info for movie 42 out of 50 on page 15
Finished getting info for movie 16 out of 50 on page 15
Finished getting info for movie 39 out of 50 on pa

Getting info for movie 41 out of 50 on page 17
Getting info for movie 42 out of 50 on page 17Getting info for movie 43 out of 50 on page 17

Getting info for movie 44 out of 50 on page 17
Getting info for movie 45 out of 50 on page 17
Getting info for movie 46 out of 50 on page 17
Getting info for movie 47 out of 50 on page 17
Getting info for movie 48 out of 50 on page 17
Getting info for movie 49 out of 50 on page 17
Getting info for movie 50 out of 50 on page 17
Finished getting info for movie 2 out of 50 on page 17
Finished getting info for movie 35 out of 50 on page 17
Finished getting info for movie 1 out of 50 on page 17
Finished getting info for movie 3 out of 50 on page 17Finished getting info for movie 45 out of 50 on page 17

Finished getting info for movie 18 out of 50 on page 17
Finished getting info for movie 17 out of 50 on page 17
Finished getting info for movie 10 out of 50 on page 17
Finished getting info for movie 14 out of 50 on page 17
Finished getting info for mov

Finished getting info for movie 40 out of 50 on page 18
Getting info for movie 1 out of 50 on page 19
Getting info for movie 2 out of 50 on page 19
Getting info for movie 3 out of 50 on page 19Getting info for movie 4 out of 50 on page 19

Getting info for movie 5 out of 50 on page 19
Getting info for movie 6 out of 50 on page 19
Getting info for movie 7 out of 50 on page 19
Getting info for movie 8 out of 50 on page 19
Getting info for movie 9 out of 50 on page 19
Getting info for movie 10 out of 50 on page 19
Getting info for movie 11 out of 50 on page 19Getting info for movie 12 out of 50 on page 19
Getting info for movie 13 out of 50 on page 19

Getting info for movie 14 out of 50 on page 19
Getting info for movie 15 out of 50 on page 19
Getting info for movie 16 out of 50 on page 19
Getting info for movie 17 out of 50 on page 19
Getting info for movie 18 out of 50 on page 19
Getting info for movie 19 out of 50 on page 19
Getting info for movie 20 out of 50 on page 19
Getting info 